<a href="https://colab.research.google.com/github/pponmathi10/sample/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import joblib
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

MODEL_PATH = "model.pkl"
VECTORIZER_PATH = "vectorizer.pkl"
DATA_PATH = "/content/AI_Resume_Screening.csv"

def preprocess_data():
    df = pd.read_csv(DATA_PATH)

    # Combine important columns into one text feature
    df["resume_text"] = (
        df["Skills"].fillna("") + " " +
        df["Education"].fillna("") + " " +
        df["Certifications"].fillna("") + " " +
        df["Job Role"].fillna("")
    )

    X = df["resume_text"]
    y = df["Recruiter Decision"]

    return X, y

def train_model():
    X, y = preprocess_data()

    vectorizer = TfidfVectorizer(
        stop_words="english",
        max_features=4000
    )
    X_vec = vectorizer.fit_transform(X)

    model = LogisticRegression(max_iter=1000)
    model.fit(X_vec, y)

    joblib.dump(model, MODEL_PATH)
    joblib.dump(vectorizer, VECTORIZER_PATH)

def load_model():
    if not os.path.exists(MODEL_PATH):
        train_model()

    model = joblib.load(MODEL_PATH)
    vectorizer = joblib.load(VECTORIZER_PATH)
    return model, vectorizer


In [4]:
import streamlit as st
import pandas as pd
from model import load_model

st.set_page_config(
    page_title="Intelligent Resume Screening System",
    layout="centered"
)

st.title("📄 Intelligent Resume Screening System")
st.subheader("Using NLP and Machine Learning")

model, vectorizer = load_model()

st.markdown("### 🔍 Enter Candidate Details")

skills = st.text_input("Skills (comma separated)")
education = st.selectbox(
    "Education",
    ["B.Sc", "B.Tech", "MBA", "M.Sc", "PhD"]
)
certifications = st.text_input("Certifications")
job_role = st.text_input("Job Role Applied For")

if st.button("🚀 Screen Resume"):
    resume_text = skills + " " + education + " " + certifications + " " + job_role

    X = vectorizer.transform([resume_text])
    prediction = model.predict(X)[0]
    confidence = max(model.predict_proba(X)[0]) * 100

    st.markdown(f"## ✅ Decision: **{prediction}**")
    st.markdown(f"### 📊 Confidence Score: **{confidence:.2f}%**")

    if prediction == "Hire":
        st.success("Candidate is suitable for the role 🎯")
    else:
        st.error("Candidate does not meet the criteria ❌")


ModuleNotFoundError: No module named 'streamlit'